<a href="https://colab.research.google.com/github/Prast667/Prit/blob/main/pp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import zipfile
from google.colab import drive

# --- KODE SETUP AWAL ---
print("Menghubungkan Google Drive...")
drive.mount('/content/drive')

zip_path = '/content/drive/MyDrive/Sinta 2.zip'
extract_dir = '/content/data_sampah/'
FOLDER_INDUK = 'Sinta 2' # NAMA FOLDER INDUK YANG BARU DITEMUKAN

# Membuat direktori ekstraksi jika belum ada
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

print(f"Mengekstrak {zip_path} ke {extract_dir}...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Ekstraksi selesai!")

# --- PENYESUAIAN PATH FINAL ---

# Path TRAIN dan TEST sekarang harus melalui FOLDER_INDUK
TRAIN_DIR = os.path.join(extract_dir, FOLDER_INDUK, 'Train')
TEST_DIR = os.path.join(extract_dir, FOLDER_INDUK, 'Test')

# Verifikasi Path
print(f"\nVerifikasi Path TRAIN: {TRAIN_DIR}")
print(f"Isi dari TRAIN_DIR: {os.listdir(TRAIN_DIR)}")
# Output harusnya ['Organik', 'Anorganik'] atau nama kelas Anda yang sebenarnya

Menghubungkan Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mengekstrak /content/drive/MyDrive/Sinta 2.zip ke /content/data_sampah/...
Ekstraksi selesai!

Verifikasi Path TRAIN: /content/data_sampah/Sinta 2/Train
Isi dari TRAIN_DIR: ['Organik', 'Anorganik']


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# =======================================================
# IMAGE DATA GENERATOR UNTUK DATA TRAINING (AUGMENTASI)
# =======================================================
train_datagen = ImageDataGenerator(
    rescale=1./255,                 # Normalisasi Wajib
    rotation_range=15,              # Rotasi antara -15 dan 15 derajat
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,           # Rotate horizontal
    vertical_flip=True,             # Rotate vertical
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

# =======================================================
# IMAGE DATA GENERATOR UNTUK DATA TESTING (HANYA NORMALISASI)
# =======================================================
test_datagen = ImageDataGenerator(rescale=1./255)


# =======================================================
# PEMUATAN DATA (GENERATOR)
# =======================================================
try:
    # Training Generator
    train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )

    # Test/Validation Generator
    test_generator = test_datagen.flow_from_directory(
        TEST_DIR,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )

    print("\n✅ Data Generator berhasil dibuat!")
    print(f"Total gambar train: {train_generator.samples}")
    print(f"Total gambar test: {test_generator.samples}")

except Exception as e:
    # Jika masih gagal, mungkin nama folder kelas (Organik/Anorganik) tidak cocok
    print(f"\n❌ GAGAL MEMUAT GENERATOR. Periksa kembali struktur folder di: {TRAIN_DIR}")
    print(f"Error: {e}")

Found 1767 images belonging to 2 classes.
Found 428 images belonging to 2 classes.

✅ Data Generator berhasil dibuat!
Total gambar train: 1767
Total gambar test: 428


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Tentukan ukuran input yang umum
IMG_SIZE = (224, 224)

def build_transfer_model(base_model_func, input_shape, model_name, learning_rate=1e-4):
    """Membangun model Transfer Learning."""

    # Memuat model pra-latih (ImageNet weights) tanpa lapisan Fully Connected
    base_model = base_model_func(weights='imagenet',
                                 include_top=False,
                                 input_shape=input_shape)

    # Membekukan lapisan dasar (Feature Extraction)
    base_model.trainable = False

    # Menambahkan lapisan klasifikasi kustom di atas
    x = base_model.output
    x = GlobalAveragePooling2D(name='global_average_pooling')(x)
    x = Dense(256, activation='relu', name='custom_dense_1')(x)
    predictions = Dense(1, activation='sigmoid', name='output_layer')(x) # Output biner

    # Menggabungkan model dasar dan lapisan baru
    model = Model(inputs=base_model.input, outputs=predictions, name=model_name)

    # Mengkompilasi model
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

# Mendefinisikan Callbacks untuk semua model
# Early Stopping: Menghentikan training jika validasi akurasi tidak membaik


# Model Checkpoint: Menyimpan model terbaik
# Ganti nama file ini saat training model yang berbeda
# checkpoint = ModelCheckpoint('best_model_NAME.h5', monitor='val_accuracy', save_best_only=True)

In [19]:
from tensorflow.keras.applications import ResNet50

print("\n--- Melatih ResNet50 ---")
resnet_model = build_transfer_model(ResNet50, IMG_SIZE + (3,), 'ResNet50_Transfer')

# Tampilkan ringkasan model
# resnet_model.summary()

resnet_history = resnet_model.fit(
    train_generator,
    epochs=50, # Jumlah epochs bisa disesuaikan
    validation_data=test_generator,
    # Tambahkan callback
    verbose=1
)


--- Melatih ResNet50 ---
Epoch 1/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 46s 637ms/step - accuracy: 0.5486 - loss: 0.6851 - val_accuracy: 0.6285 - val_loss: 0.6407
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 27s 469ms/step - accuracy: 0.6652 - loss: 0.6299 - val_accuracy: 0.6869 - val_loss: 0.6088
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 25s 440ms/step - accuracy: 0.6742 - loss: 0.6202 - val_accuracy: 0.7009 - val_loss: 0.5913
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 25s 446ms/step - accuracy: 0.6822 - loss: 0.5929 - val_accuracy: 0.6869 - val_loss: 0.5837
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 26s 461ms/step - accuracy: 0.6760 - loss: 0.5878 - val_accuracy: 0.6916 - val_loss: 0.5724
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 26s 461ms/step - accuracy: 0.6968 - loss: 0.5778 - val_accuracy: 0.7009 - val_loss: 0.5676
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 26s 459ms/step - accuracy: 0.6947 - loss: 0.5770 - val_accuracy: 0.7407 - val_loss: 0.5410
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 26s 459ms/step - accuracy: 0.7170

In [20]:
from tensorflow.keras.applications import EfficientNetB0

print("\n--- Melatih EfficientNetB0 ---")
efficientnet_model = build_transfer_model(EfficientNetB0, IMG_SIZE + (3,), 'EfficientNetB0_Transfer')

efficientnet_history = efficientnet_model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    verbose=1
)


--- Melatih EfficientNetB0 ---
Epoch 1/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 65s 787ms/step - accuracy: 0.5597 - loss: 0.6948 - val_accuracy: 0.5374 - val_loss: 0.6906
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 24s 430ms/step - accuracy: 0.5368 - loss: 0.6928 - val_accuracy: 0.5374 - val_loss: 0.6917
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 24s 431ms/step - accuracy: 0.5574 - loss: 0.6889 - val_accuracy: 0.5374 - val_loss: 0.6900
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 24s 434ms/step - accuracy: 0.5530 - loss: 0.6893 - val_accuracy: 0.5374 - val_loss: 0.6943
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 24s 431ms/step - accuracy: 0.5538 - loss: 0.6905 - val_accuracy: 0.5374 - val_loss: 0.6901
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 23s 413ms/step - accuracy: 0.5359 - loss: 0.6921 - val_accuracy: 0.5374 - val_loss: 0.6901
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 24s 416ms/step - accuracy: 0.5571 - loss: 0.6884 - val_accuracy: 0.5374 - val_loss: 0.6899
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 24s 425ms/step - accuracy: 

In [21]:
from tensorflow.keras.applications import DenseNet121

print("\n--- Melatih DenseNet121 ---")
densenet_model = build_transfer_model(DenseNet121, IMG_SIZE + (3,), 'DenseNet121_Transfer')

densenet_history = densenet_model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    verbose=1
)


--- Melatih DenseNet121 ---
Epoch 1/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.6994 - loss: 0.5623 - val_accuracy: 0.9673 - val_loss: 0.1446
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 25s 453ms/step - accuracy: 0.9732 - loss: 0.1350 - val_accuracy: 0.9743 - val_loss: 0.0853
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 25s 455ms/step - accuracy: 0.9722 - loss: 0.0909 - val_accuracy: 0.9790 - val_loss: 0.0697
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 25s 446ms/step - accuracy: 0.9856 - loss: 0.0589 - val_accuracy: 0.9860 - val_loss: 0.0608
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 25s 452ms/step - accuracy: 0.9825 - loss: 0.0557 - val_accuracy: 0.9860 - val_loss: 0.0579
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 26s 457ms/step - accuracy: 0.9820 - loss: 0.0496 - val_accuracy: 0.9836 - val_loss: 0.0557
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 25s 451ms/step - accuracy: 0.9880 - loss: 0.0371 - val_accuracy: 0.9836 - val_loss: 0.0562
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 26s 457ms/step - accuracy: 0.9924

In [22]:
from tensorflow.keras.applications import MobileNetV2

print("\n--- Melatih MobileNetV2 ---")
mobilenet_model = build_transfer_model(MobileNetV2, IMG_SIZE + (3,), 'MobileNetV2_Transfer')

mobilenet_history = mobilenet_model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    verbose=1
)


--- Melatih MobileNetV2 ---
Epoch 1/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 44s 631ms/step - accuracy: 0.8158 - loss: 0.4146 - val_accuracy: 0.9766 - val_loss: 0.1047
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 23s 408ms/step - accuracy: 0.9756 - loss: 0.0926 - val_accuracy: 0.9813 - val_loss: 0.0732
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 24s 424ms/step - accuracy: 0.9843 - loss: 0.0654 - val_accuracy: 0.9813 - val_loss: 0.0628
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 24s 427ms/step - accuracy: 0.9840 - loss: 0.0523 - val_accuracy: 0.9720 - val_loss: 0.0801
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 24s 427ms/step - accuracy: 0.9855 - loss: 0.0427 - val_accuracy: 0.9766 - val_loss: 0.0667
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 24s 428ms/step - accuracy: 0.9882 - loss: 0.0373 - val_accuracy: 0.9650 - val_loss: 0.0903
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 23s 408ms/step - accuracy: 0.9874 - loss: 0.0394 - val_accuracy: 0.9766 - val_loss: 0.0658
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 24s 434ms/step - accuracy: 0.9

In [23]:
from tensorflow.keras.applications import VGG16

# VGG16 cenderung lebih lambat dan membutuhkan banyak memori
print("\n--- Melatih VGG16 ---")
vgg16_model = build_transfer_model(VGG16, IMG_SIZE + (3,), 'VGG16_Transfer')

vgg16_history = vgg16_model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    verbose=1
)


--- Melatih VGG16 ---
Epoch 1/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 33s 528ms/step - accuracy: 0.6576 - loss: 0.6571 - val_accuracy: 0.8855 - val_loss: 0.4794
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 28s 498ms/step - accuracy: 0.8743 - loss: 0.4549 - val_accuracy: 0.9276 - val_loss: 0.3429
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 28s 495ms/step - accuracy: 0.8979 - loss: 0.3458 - val_accuracy: 0.9369 - val_loss: 0.2682
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 28s 500ms/step - accuracy: 0.9197 - loss: 0.2860 - val_accuracy: 0.9416 - val_loss: 0.2273
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 28s 495ms/step - accuracy: 0.9284 - loss: 0.2500 - val_accuracy: 0.9533 - val_loss: 0.1985
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 28s 501ms/step - accuracy: 0.9279 - loss: 0.2351 - val_accuracy: 0.9579 - val_loss: 0.1799
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 28s 497ms/step - accuracy: 0.9228 - loss: 0.2054 - val_accuracy: 0.9579 - val_loss: 0.1677
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 28s 497ms/step - accuracy: 0.9268 - 

In [4]:
from tensorflow.keras.applications import NASNetMobile

print("\n--- Melatih NASNetMobile ---")
nasnet_model = build_transfer_model(NASNetMobile, IMG_SIZE + (3,), 'NASNetMobile_Transfer')

nasnet_history = nasnet_model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    verbose=1
)


--- Melatih NASNetMobile ---


NameError: name 'build_transfer_model' is not defined

In [3]:
import pandas as pd

results = {
    'Model': ['ResNet50', 'EfficientNetB0', 'DenseNet121', 'MobileNetV2', 'NASNetMobile', 'VGG16'],
    'Validation Accuracy': [
        max(resnet_history.history['val_accuracy']),
        max(efficientnet_history.history['val_accuracy']),
        max(densenet_history.history['val_accuracy']),
        max(mobilenet_history.history['val_accuracy']),
        max(nasnet_history.history['val_accuracy']),
        max(vgg16_history.history['val_accuracy'])
    ],
    'Training Loss (Last Epoch)': [
        resnet_history.history['loss'][-1],
        efficientnet_history.history['loss'][-1],
        densenet_history.history['loss'][-1],
        mobilenet_history.history['loss'][-1],
        nasnet_history.history['loss'][-1],
        vgg16_history.history['loss'][-1]
    ]
}

df_results = pd.DataFrame(results)
print("\n--- Hasil Perbandingan Model ---")
print(df_results.sort_values(by='Validation Accuracy', ascending=False))

NameError: name 'resnet_history' is not defined

In [ ]:
import matplotlib.pyplot as plt

# Kumpulkan semua history object ke dalam satu dictionary
# GANTI DENGAN NAMA HISTORY OBJECT ANDA
all_histories = {
    "ResNet50": resnet_history,
    "EfficientNetB0": efficientnet_history,
    "DenseNet121": densenet_history,
    "MobileNetV2": mobilenet_history,
    "NASNetMobile": nasnet_history,
    "VGG16": vgg16_history
}

plt.figure(figsize=(15, 6))

# --- Plot Validation Accuracy ---
plt.subplot(1, 2, 1)
for name, history in all_histories.items():
    plt.plot(history.history['val_accuracy'], label=name)
plt.title('Validation Accuracy Comparison')
plt.ylabel('Validation Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.grid(True)

# --- Plot Validation Loss ---
plt.subplot(1, 2, 2)
for name, history in all_histories.items():
    plt.plot(history.history['val_loss'], label=name)
plt.title('Validation Loss Comparison')
plt.ylabel('Validation Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Kumpulkan semua model object ke dalam satu dictionary
# GANTI DENGAN NAMA MODEL OBJECT ANDA
all_models = {
    "ResNet50": resnet_model,
    "EfficientNetB0": efficientnet_model,
    "DenseNet121": densenet_model,
    "MobileNetV2": mobilenet_model,
    "NASNetMobile": nasnet_model,
    "VGG16": vgg16_model
}

# --- 1. Ambil True Labels (Label Sebenarnya) ---
# Mengatur ulang generator untuk memastikan urutan data yang benar
test_generator.reset()
true_labels = test_generator.classes
class_names = list(test_generator.class_indices.keys())
print(f"Nama Kelas: {class_names}")

# --- 2. Fungsi untuk Membuat Confusion Matrix ---
def plot_confusion_matrix(model, model_name, generator, true_labels, class_names):

    # 1. Prediksi Probabilitas
    y_pred_proba = model.predict(generator)

    # 2. Konversi Probabilitas ke Label Biner (0 atau 1)
    # Karena ini klasifikasi biner dengan sigmoid, kita gunakan threshold 0.5
    y_pred_binary = (y_pred_proba > 0.5).astype("int32")

    # 3. Hitung Confusion Matrix
    cm = confusion_matrix(true_labels, y_pred_binary)

    # 4. Visualisasi
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix: {model_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

    # 5. Tampilkan Classification Report (Precision, Recall, F1-Score)
    print(f"\nClassification Report for {model_name}:")
    print(classification_report(true_labels, y_pred_binary, target_names=class_names))

# --- 3. Eksekusi untuk Setiap Model ---
for name, model in all_models.items():
    print(f"\n===========================================")
    print(f"Menganalisis Kinerja: {name}")

    # Reset generator sebelum prediksi untuk memastikan urutan data
    test_generator.reset()

    # Plot dan Report
    plot_confusion_matrix(model, name, test_generator, true_labels, class_names)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import time

# Asumsi: all_models dan test_generator sudah didefinisikan dari langkah sebelumnya

# Kumpulkan semua history object dan model object
all_histories = {
    "ResNet50": resnet_history,
    "EfficientNet": efficientnet_history, # Dianggap EfficientNetB0
    "DenseNet": densenet_history, # Dianggap DenseNet121
    "MobileNetV2": mobilenet_history,
    "NASNetMobile": nasnet_history,
    "VGG16": vgg16_history
}
all_models = {
    "ResNet50": resnet_model,
    "EfficientNet": efficientnet_model,
    "DenseNet": densenet_model,
    "MobileNetV2": mobilenet_model,
    "NASNetMobile": nasnet_model,
    "VGG16": vgg16_model
}

# Inisialisasi daftar untuk menampung data
data_rows = []
NUM_INFERENCE_TESTS = 100 # Jumlah gambar yang akan diuji untuk Inference Speed
input_shape = IMG_SIZE + (3,)

# --- Loop Melalui Setiap Model ---
for name, model in all_models.items():
    print(f"Memproses metrik untuk: {name}...")

    # A. Prediksi dan Laporan Klasifikasi
    test_generator.reset()
    true_labels = test_generator.classes

    # Dapatkan prediksi probabilitas
    y_pred_proba = model.predict(test_generator)
    y_pred_binary = (y_pred_proba > 0.5).astype("int32")

    # Hitung Classification Report
    report = classification_report(true_labels, y_pred_binary, output_dict=True, zero_division=0)

    # Karena ini biner, kita bisa gunakan rata-rata 'weighted avg' dari report
    accuracy = report['accuracy']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    f1_score = report['weighted avg']['f1-score']

    # B. Waktu Pelatihan (Training Time) dan Epoch

    # Early Stopping menghentikan training, jadi kita ambil epoch terakhir yang sebenarnya
    epochs_used = len(all_histories[name].history['loss'])
    # Colab tidak memberikan waktu per epoch, jadi kita akan hitung total waktu
    # NOTE: Anda harus MENCATAT MANUAL total waktu training dari Colab Output!
    # Di sini, kita akan menggunakan nilai placeholder yang realistis:

    # Placeholder Waktu (GANTI dengan data manual Anda)
    if name == 'MobileNetV2':
        training_time = "5m:10s"
    elif name == 'EfficientNet':
        training_time = "6m:40s"
    elif name == 'ResNet50':
        training_time = "8m:50s"
    elif name == 'VGG16':
        training_time = "10m:20s"
    else:
        training_time = f"{epochs_used} E" # Placeholder jika waktu tidak tersedia

    # C. Inference Speed (Waktu Prediksi)

    # Buat data dummy untuk inferensi
    dummy_input = np.random.rand(NUM_INFERENCE_TESTS, input_shape[0], input_shape[1], input_shape[2])

    start_time = time.time()
    # Lakukan prediksi pada data dummy untuk mendapatkan waktu yang akurat
    model.predict(dummy_input, verbose=0)
    end_time = time.time()

    # Kecepatan Inferensi: Waktu per gambar (detik/gambar)
    inference_speed = (end_time - start_time) / NUM_INFERENCE_TESTS

    # Tambahkan baris data
    data_rows.append({
        'Model': name,
        'Accuracy': f"{accuracy:.4f}",
        'Precision': f"{precision:.4f}",
        'Recall': f"{recall:.4f}",
        'F1-score': f"{f1_score:.4f}",
        'Epoch': epochs_used,
        'Training Time (Manual)': training_time,
        'Inference Speed (s/img)': f"{inference_speed:.4f}"
    })

# Buat DataFrame
df_comparison = pd.DataFrame(data_rows)
df_comparison = df_comparison.sort_values(by='Accuracy', ascending=False)

In [ ]:
print("\n=========================================================================")
print("Tabel 1. Perbandingan Metrik Kinerja Model Klasifikasi Sampah")
print("=========================================================================")
print(df_comparison.to_markdown(index=False, numalign="left", stralign="left"))

In [ ]:
import matplotlib.pyplot as plt

# Asumsi: all_histories sudah terdefinisi dari langkah pelatihan model Anda.
# all_histories = {
#     "ResNet50": resnet_history,
#     "EfficientNet": efficientnet_history,
#     "DenseNet": densenet_history,
#     "MobileNetV2": mobilenet_history,
#     "NASNetMobile": nasnet_history,
#     "VGG16": vgg16_history
# }

# Definisikan palet warna dan gaya garis untuk konsistensi
colors = ['blue', 'red', 'green', 'purple', 'orange', 'brown']
# NOTE: Gunakan 6 warna yang konsisten untuk 6 model

plt.figure(figsize=(16, 6))

# =======================================================
# Plot Kiri: Loss dan Validation Loss
# =======================================================
plt.subplot(1, 2, 1)
plt.title('Training and Validation Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
legend_handles_loss = []

# Loop untuk plotting
for i, (name, history) in enumerate(all_histories.items()):
    color = colors[i % len(colors)]

    # Training Loss (Garis Solid)
    line1, = plt.plot(history.history['loss'],
                      label=f"{name} - Loss",
                      color=color, linestyle='-')

    # Validation Loss (Garis Putus-Putus)
    line2, = plt.plot(history.history['val_loss'],
                      label=f"{name} - Val Loss",
                      color=color, linestyle='--')

    # Simpan handle untuk legend Loss
    legend_handles_loss.extend([line1, line2])

# Penempatan LEGEND LOSS di luar plot:
# bbox_to_anchor=(1.05, 1) menempatkan legend di sudut kanan atas plot
plt.legend(handles=legend_handles_loss,
           loc='upper left',
           bbox_to_anchor=(1.05, 1),
           fontsize='small',
           title="Legend Loss")


# =======================================================
# Plot Kanan: Accuracy dan Validation Accuracy
# =======================================================
plt.subplot(1, 2, 2)
plt.title('Training and Validation Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid(True)
legend_handles_acc = []

# Loop untuk plotting
for i, (name, history) in enumerate(all_histories.items()):
    color = colors[i % len(colors)]

    # Training Accuracy (Garis Solid)
    line3, = plt.plot(history.history['accuracy'],
                      label=f"{name} - Accuracy",
                      color=color, linestyle='-')

    # Validation Accuracy (Garis Putus-Putus)
    line4, = plt.plot(history.history['val_accuracy'],
                      label=f"{name} - Val Accuracy",
                      color=color, linestyle='--')

    # Simpan handle untuk legend Accuracy
    legend_handles_acc.extend([line3, line4])

# Penempatan LEGEND ACCURACY di luar plot:
# bbox_to_anchor=(1.05, 1) menempatkan legend di sudut kanan atas plot
plt.legend(handles=legend_handles_acc,
           loc='upper left',
           bbox_to_anchor=(1.05, 1),
           fontsize='small',
           title="Legend Accuracy")

plt.tight_layout(rect=[0, 0, 0.85, 1]) # Sesuaikan area plot agar legend muat
plt.show()